# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import csv
# import pyodbc
import os
import warnings

# Constants

In [2]:
# Define the years for each dataframe
years = [2017, 2018, 2019, 2021, 2022]

# Intialize Useful Functions

In [3]:
def safe_convert(val):
    try:
        return int(val)
    except ValueError:
        print(f"Value {val} can't be converted to int")
        return None
    
def import_mdb(MDBs, DRV, PWD, NAMES):
    
    databases = {}
    
    for MDB, NAME in zip(MDBs, NAMES):
        # connect to db
        con = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV,MDB,PWD))
        cur = con.cursor()

        # List all tables in the database
        tables = list(map(lambda t: t.table_name, con.cursor().tables(tableType='TABLE')))

        # Initialize an empty dictionary to hold your dataframes and databases
        database = {}

        # Try to read each table one by one
        for table in tables:
            try:
                df = pd.read_sql(f'SELECT * FROM [{table}]', con)  # enclose table name in brackets
                database[table] = df
                print(f"Successfully read table: {table}")
            except Exception as e:
                print(f"Failed to read table: {table}")
                print(f"Error: {e}")
        databases[NAME] = database
        
    return databases

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore', 'pandas only support SQLAlchemy connectable.*')
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)
# warnings.filterwarnings('ignore', category=pd.core.common.SettingWithCopyWarning)

# Import Main Data

In [4]:
# Main Data Filepaths WINDOWS
# MAIN_PATH = [
#                '../data/SRC2019/SRC2019.mdb;',
#                '../data/SRC2021/SRC2021.mdb;',
#                '../data/SRC2022/SRC2022.mdb;',
#               ]

# Main Data Filepaths MAC
MAIN_PATH = [
               '../../data/SRC2017/ARE2017.csv',
               '../../data/SRC2018/ARE2018.csv',
               '../../data/SRC2019/ARE2019.csv',
               '../../data/SRC2021/ARE2021.csv',
               '../../data/SRC2022/ARE2022.CSV',
              ]
MAIN_NAMES = [
               'MAIN2017',
               'MAIN2018',
               'MAIN2019',
               'MAIN2021',
               'MAIN2022',
              ]

main_data = {}
for name, filepath in zip(MAIN_NAMES, MAIN_PATH):
    main_data[name] = {'Annual Regents Exams': pd.read_csv(filepath, thousands=',')}

# Main data Windows
# main_data = import_mdb(MAIN_PATH, DRV, PWD, MAIN_NAMES)

# Combine Main Data

### Remove Districts, Keep Schools Common Across Databases

In [5]:
common_entity_ids = set(main_data['MAIN2019']['Annual Regents Exams']['ENTITY_CD']) 

for database in main_data:
    current_data = main_data[database]['Annual Regents Exams']
    current_data['ENTITY_CD'] = current_data['ENTITY_CD'].apply(safe_convert)
    current_data = current_data[~current_data['ENTITY_CD'].astype(str).str.endswith('0000')]

    common_entity_ids = set(current_data['ENTITY_CD']) & common_entity_ids
    
for database in main_data:
    current_data = main_data[database]['Annual Regents Exams']
    main_data[database]['Annual Regents Exams'] = current_data[current_data['ENTITY_CD'].isin(common_entity_ids)]

In [6]:
for database in main_data:
    current_data = main_data[database]['Annual Regents Exams']
    main_data[database] = {subject: current_data[current_data['SUBJECT'] == subject] for subject in current_data['SUBJECT'].unique()}

In [7]:
old_to_new = {
    'REG_PHYS_PS':'Regents Phy Set/Physics',
    'REG_NF_GLHIST':'Regents NF Global History',
    'REG_COMENG':'Regents Common Core English Language Art', 
    'REG_ESCI_PS':'Regents Phy Set/Earth Sci',
    'REG_CHEM_PS':'Regents Phy Set/Chemistry', 
    'REG_COMALG1':'Regents Common Core Algebra I', 
    'REG_COMGEOM':'Regents Common Core Geometry', 
    'REG_LENV':'Regents Living Environment',
    'REG_USHG_RV':"Regents US History&Gov't"
}

new_to_old = {}

for key in old_to_new:
    new_to_old[old_to_new[key]] = key

tests = (set(old_to_new[test] for test in old_to_new) 
         & set(test for test in main_data['MAIN2021']) 
         & set(test for test in main_data['MAIN2022']))



### Calculate Demographic Percentages

In [8]:
for year, database in zip(years, MAIN_NAMES):
    current_df = None
    current_df = (main_data[database]['Regents Common Core English Language Art'] 
                  if 'Regents Common Core English Language Art' in main_data[database]
                else main_data[database][new_to_old['Regents Common Core English Language Art']])

    # Filter the DataFrame to only include rows where SUBGROUP_NAME == 'All Students'
    total_students_df = current_df[(current_df['SUBGROUP_NAME'] == 'All Students') & (current_df['YEAR'] == year)][['ENTITY_CD', 'TESTED', 'YEAR']]
    print("checkpoint 1")

    # Merge the total students for 'All Students' back into the original DataFrame
    current_df = pd.merge(current_df, total_students_df, on=['ENTITY_CD', 'YEAR'], how='left', suffixes=('', '_total'))
    print("checkpoint 2")

    # List of subgroups of interest
    KEPT_SUBGROUPS = ['Male', 'Female', 'White', 'Hispanic or Latino', 'Black or African American', 'Asian or Native Hawaiian/Other Pacific Islander','Economically Disadvantaged']

    # List to store DataFrames
    df_list = []
    columns = "ENTITY_CD  ENTITY_NAME YEAR SUBJECT TESTED TESTED_total NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF PER_PROF"
    columns = columns.split()

    # Get ENTITY_NAME for 'All Students' subgroup
    multiple_df = current_df[current_df['SUBGROUP_NAME'] == 'All Students'][columns]
    print("checkpoint 3")

    # Loop over each subgroup and calculate percentage
    for subgroup in KEPT_SUBGROUPS:
        temp_df = current_df[(current_df['SUBGROUP_NAME'] == subgroup) & (current_df['YEAR'] == year)].copy()
        subgroup = subgroup.upper()
        temp_df[subgroup + '_PCT'] = temp_df['TESTED'] / temp_df['TESTED_total'] * 100
        temp_df = temp_df[['ENTITY_CD', subgroup + '_PCT']]  # Keep 'ENTITY_CD' in each temp_df
        df_list.append(temp_df)
    print("checkpoint 4")

    # Merge all DataFrames on ENTITY_CD
    result_df = multiple_df
    for temp_df in df_list:
        result_df = result_df.merge(temp_df, on='ENTITY_CD', how='outer')
    print("checkpoint 5")


    # Drop observations where TESTED is less than 4
    result_df = result_df[result_df['TESTED_total'] >= 2]
    print("checkpoint 6")

    # Fill NaN values with 0
    result_df = result_df.fillna(0)
    print("checkpoint 7")

    cols = "NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF PER_PROF".split()
    for col in cols:
        result_df[col] = result_df[col].replace('s', 0)
    print("checkpoint 7")
    if 'Regents Common Core English Language Art' in main_data[database]:
        main_data[database]['Regents Common Core English Language Art'] = result_df  # store results in new dictionary instead of main_data
    else:
        main_data[database][new_to_old['Regents Common Core English Language Art']] = result_df

checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
checkpoint 6
checkpoint 7
checkpoint 7
checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
checkpoint 6
checkpoint 7
checkpoint 7
checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
checkpoint 6
checkpoint 7
checkpoint 7
checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
checkpoint 6
checkpoint 7
checkpoint 7
checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
checkpoint 6
checkpoint 7
checkpoint 7


### Concatenate All Dataframes

In [9]:
final_data = pd.concat([
    main_data['MAIN2017'][new_to_old['Regents Common Core English Language Art']],
    main_data['MAIN2018'][new_to_old['Regents Common Core English Language Art']],
    main_data['MAIN2019'][new_to_old['Regents Common Core English Language Art']], 
    main_data['MAIN2021']['Regents Common Core English Language Art'], 
    main_data['MAIN2022']['Regents Common Core English Language Art']])
                        

In [10]:
final_data['SUBJECT'] = 'Regents Common Core English Language Art'


In [11]:
# Convert 'YEAR' in final_data to int
final_data['YEAR'] = final_data['YEAR'].astype(int)

# # Select specific columns from dropout_df
# dropout_subset = dropout_df[['aggregation_code', 'report_school_year', 'dropout_pct']]

# # Merge dropout_df with final_data
# final_data = pd.merge(final_data, dropout_subset, left_on=['ENTITY_CD', 'YEAR'], right_on=['aggregation_code', 'report_school_year'], how='left')

# # Replace '-' with np.nan
# final_data['dropout_pct'] = final_data['dropout_pct'].replace('-', np.nan)

# # Remove '%' from 'dropout_pct' and convert to float
# final_data['dropout_pct'] = final_data['dropout_pct'].str.rstrip('%').astype('float')

# final_data['dropout_pct'] = final_data['dropout_pct'].fillna(0)

In [12]:
final_data

,ENTITY_CD,ENTITY_NAME,YEAR,SUBJECT,TESTED,TESTED_total,NUM_LEVEL1,PER_LEVEL1,NUM_LEVEL2,PER_LEVEL2,NUM_LEVEL3,PER_LEVEL3,NUM_LEVEL4,PER_LEVEL4,NUM_LEVEL5,PER_LEVEL5,NUM_PROF,PER_PROF,MALE_PCT,FEMALE_PCT,WHITE_PCT,HISPANIC OR LATINO_PCT,BLACK OR AFRICAN AMERICAN_PCT,ASIAN OR NATIVE HAWAIIAN/OTHER PACIFIC ISLANDER_PCT,ECONOMICALLY DISADVANTAGED_PCT
2,522001040003,STILLWATER MIDDLE SCHOOL HIGH SCHOOL,2017,Regents Common Core English Language Art,103,103.0,11,11,8,08,18,17,18,17,48,47,84.0,81.0,44.660194,55.339806,95.145631,0.970874,1.941748,0.000000,28.155340
5,141901060008,LANCASTER HIGH SCHOOL,2017,Regents Common Core English Language Art,525,525.0,10,02,11,02,52,10,60,11,392,75,504.0,96.0,49.333333,50.666667,93.142857,3.428571,0.761905,0.952381,19.238095
8,580224030008,PATCHOGUE-MEDFORD HIGH SCHOOL,2017,Regents Common Core English Language Art,637,637.0,33,05,33,05,105,16,117,18,349,55,571.0,89.0,51.020408,48.979592,56.671900,33.594976,5.494505,3.610675,48.665620
10,1,NYC Public Schools,2017,Regents Common Core English Language Art,87603,87603.0,12792,15,6699,08,21341,24,16048,18,30723,35,68112.0,77.0,51.699143,48.300857,12.915083,39.171033,28.134881,17.557618,73.053434
14,2,Large Cities,2017,Regents Common Core English Language Art,8229,8229.0,1915,23,824,10,2228,27,1244,15,2018,25,5490.0,67.0,50.710900,49.289100,14.910682,29.493256,45.643456,8.518654,78.478551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2498,331700010189,PS 189 BILINGUAL CENTER,2022,Regents Common Core English Language Art,24,24.0,0,00,0,00,9,38,10,42,5,21,24,100,37.500000,62.500000,0.000000,20.833333,75.000000,0.000000,87.500000
2499,332200860978,BROOKLYN DREAMS CHARTER SCHOOL,2022,Regents Common Core English Language Art,13,13.0,0,00,1,08,3,23,3,23,6,46,12,92,61.538462,38.461538,0.000000,0.000000,69.230769,30.769231,84.615385
2500,342500011252,QUEENS SCHOOL OF INQUIRY,2022,Regents Common Core English Language Art,110,110.0,14,13,6,05,24,22,14,13,52,47,90,82,55.454545,44.545455,16.363636,25.454545,11.818182,44.545455,68.181818
2502,353100010034,IS 34 TOTTENVILLE,2022,Regents Common Core English Language Art,52,52.0,0,00,0,00,10,19,22,42,20,38,52,100,30.769231,69.230769,78.846154,13.461538,0.000000,5.769231,21.153846


# Import Virtual Mode Data

In [13]:
virtual = pd.read_csv("../../data/New_York_Schools_LearningModelData_Final.csv", thousands=',')
virtual['Charter'] = virtual['Charter'].replace({'Yes': 1, 'No': 0})

In [14]:
virtual = virtual[virtual['TimePeriodStart'].str.endswith(('21', '22'))]
virtual.head()

,StateName,StateAbbrev,DataLevel,Charter,SchoolName,SchoolType,NCESSchoolID,StateAssignedSchoolID,DistrictName,DistrictType,NCESDistrictID,StateAssignedDistrictID,TimePeriodInterval,TimePeriodStart,TimePeriodEnd,EnrollmentTotal,LearningModel,LearningModelGrK5,LearningModelGr68,LearningModelGr912,LearningModelStateCat,LearningModelStateCatGrK5,LearningModelStateCatGr68,LearningModelStateCatGr912,EnrollmentInPerson,EnrollmentHybrid,EnrollmentVirtual,StaffCount,StaffCountInPerson
12,New York,NY,School,0,James A Green High School,Regular school,360000104498,211003040002,Dolgeville Central School District,Regular local school district,3600001,211003040000,Weekly,1/3/21,1/9/21,378,Virtual,NaN,NaN,NaN,Remote Only,NaN,NaN,NaN,79.0,0.0,299.0,81.0,45.0
13,New York,NY,School,0,James A Green High School,Regular school,360000104498,211003040002,Dolgeville Central School District,Regular local school district,3600001,211003040000,Weekly,1/10/21,1/16/21,378,Hybrid,NaN,NaN,NaN,Hybrid,NaN,NaN,NaN,0.0,237.0,141.0,81.0,81.0
14,New York,NY,School,0,James A Green High School,Regular school,360000104498,211003040002,Dolgeville Central School District,Regular local school district,3600001,211003040000,Weekly,1/17/21,1/23/21,378,Hybrid,NaN,NaN,NaN,Hybrid,NaN,NaN,NaN,0.0,226.0,152.0,81.0,76.0
15,New York,NY,School,0,James A Green High School,Regular school,360000104498,211003040002,Dolgeville Central School District,Regular local school district,3600001,211003040000,Weekly,1/24/21,1/30/21,378,Hybrid,NaN,NaN,NaN,Hybrid,NaN,NaN,NaN,0.0,229.0,149.0,81.0,81.0
16,New York,NY,School,0,James A Green High School,Regular school,360000104498,211003040002,Dolgeville Central School District,Regular local school district,3600001,211003040000,Weekly,1/31/21,2/6/21,378,Hybrid,NaN,NaN,NaN,Hybrid,NaN,NaN,NaN,0.0,230.0,148.0,81.0,80.0


In [15]:
# Convert the date columns to datetime format.
virtual['TimePeriodStart'] = pd.to_datetime(virtual['TimePeriodStart'])
virtual['TimePeriodEnd'] = pd.to_datetime(virtual['TimePeriodEnd'])

# Create a new column for year
virtual['YEAR'] = virtual['TimePeriodStart'].dt.year

# Fill in any missing values in LearningModel with 'InPerson'
virtual['LearningModel'] = virtual['LearningModel'].fillna('InPerson')

# Replace 'In-person' with 'InPerson'
virtual['LearningModel'] = virtual['LearningModel'].replace('In-person', 'InPerson')

# Calculate the number of days for each row
virtual['Days'] = (virtual['TimePeriodEnd'] - virtual['TimePeriodStart']).dt.days

# Group by School, Year, LearningModel, and Charter and sum the number of days
grouped = virtual.groupby(['StateAssignedSchoolID', 'YEAR', 'LearningModel', 'Charter', 'DistrictName'])['Days'].sum().reset_index()

# Pivot the data so we have separate columns for each learning model
pivot = grouped.pivot_table(index=['StateAssignedSchoolID', 'YEAR', 'Charter', 'DistrictName'], columns='LearningModel', values='Days', fill_value=0)

# # Group by School, Year, and LearningModel and sum the number of days
# grouped = virtual.groupby(['StateAssignedSchoolID', 'YEAR', 'LearningModel'])['Days'].sum().reset_index()

# # Pivot the data so we have separate columns for each learning model
# pivot = grouped.pivot_table(index=['StateAssignedSchoolID', 'YEAR'], columns='LearningModel', values='Days', fill_value=0)

# Reset the index
pivot.reset_index(inplace=True)

# Calculate the total days in each year
pivot['TotalDays'] = pivot['Virtual'] + pivot['Hybrid'] + pivot['InPerson']

# Calculate the percentage of days that are virtual and hybrid for each year
pivot['VirtualPercent'] = pivot['Virtual'] / pivot['TotalDays']
pivot['HybridPercent'] = pivot['Hybrid'] / pivot['TotalDays']

# Calculate the score for each year
pivot['Score'] = (pivot['Virtual'] + 0.5 * pivot['Hybrid']) / pivot['TotalDays']

# Reset the column names after pivot
pivot.columns.name = None

/var/folders/8y/bjy80p890fg6hgk57b8wz5d40000gn/T/ipykernel_33236/4097726366.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  virtual['TimePeriodStart'] = pd.to_datetime(virtual['TimePeriodStart'])
/var/folders/8y/bjy80p890fg6hgk57b8wz5d40000gn/T/ipykernel_33236/4097726366.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  virtual['TimePeriodEnd'] = pd.to_datetime(virtual['TimePeriodEnd'])


In [16]:
pivot = pivot.drop(columns=['InPerson', 'Hybrid', 'Virtual', 'TotalDays'])
pivot

,StateAssignedSchoolID,YEAR,Charter,DistrictName,VirtualPercent,HybridPercent,Score
0,10100010014,2021,0,Albany City School District,0.04,0.88,0.48
1,10100010016,2021,0,Albany City School District,0.04,0.88,0.48
2,10100010018,2021,0,Albany City School District,0.04,0.88,0.48
3,10100010019,2021,0,Albany City School District,0.04,0.88,0.48
4,10100010023,2021,0,Albany City School District,0.04,0.88,0.48
...,...,...,...,...,...,...,...
4398,680601060001,2021,0,Penn Yan Central School District,0.08,0.00,0.08
4399,680601060002,2021,0,Penn Yan Central School District,0.08,0.00,0.08
4400,680601060005,2021,0,Penn Yan Central School District,0.08,0.00,0.08
4401,680801040001,2021,0,Dundee Central School District,0.08,0.00,0.08


In [17]:
# Merge dropout_df with final_data
final_data = pd.merge(final_data, pivot, left_on=['ENTITY_CD', 'YEAR'], right_on=['StateAssignedSchoolID', 'YEAR'], how='left')

In [18]:
# final_data = final_data.drop(columns=[col for col in "ENTITY_NAME TESTED_total NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF aggregation_code report_school_year StateAssignedSchoolID MALE_PCT FEMALE_PCT dropout_pct SUBJECT".split()])
final_data = final_data.drop(columns=[col for col in "ENTITY_NAME TESTED_total NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF  StateAssignedSchoolID MALE_PCT FEMALE_PCT  SUBJECT".split()])

In [19]:
# Rename multiple columns
final_data = final_data.rename(columns={'ENTITY_CD': 'schoolcode', 
                        'TESTED': 'totalenroll', 
                        'PER_PROF': 'elapass', 
                        'WHITE_PCT': 'white',
                        'HISPANIC OR LATINO_PCT': 'hispanic',     
                        'BLACK OR AFRICAN AMERICAN_PCT': 'black',         
                        'ASIAN OR NATIVE HAWAIIAN/OTHER PACIFIC ISLANDER_PCT': 'asian',                     
                        'ECONOMICALLY DISADVANTAGED_PCT': 'lowincome',                                         
                        'VirtualPercent': 'virtualper',    
                        'HybridPercent': 'hybridper',    
                        'Score': 'schoolmode',    
                        'YEAR': 'year',    
                        'Charter': "charter",
                        'DistrictName': "district",
                                       })

In [20]:
# Identify 'schoolcode' values of rows in 2021 where 'schoolmode' is NaN
schoolcodes_to_remove = final_data.loc[(final_data['year'] == 2021) & (final_data['schoolmode'].isna()), 'schoolcode'].unique()

# Remove all rows with those 'schoolcode' values
final_data = final_data.loc[~final_data['schoolcode'].isin(schoolcodes_to_remove)]

In [21]:
final_data = final_data.fillna(0)
final_data["schoolcode"] = final_data["schoolcode"].astype(str)
final_data['districtcode'] = final_data['schoolcode'].str[:-4].astype(str)
final_data['countycode'] = final_data['schoolcode'].str[:-10].astype(str)

final_cols = [
'schoolcode',
'districtcode',
'countycode',
 'year',
 'charter',
 'elapass',
 'schoolmode',
 'virtualper',
 'hybridper',
 'totalenroll',
 'lowincome',
 'white',
 'black',
 'hispanic',
 'asian',
      ]

In [22]:
final_data = final_data[final_cols]
rounding_cols = [
 'schoolmode',
 'virtualper',
 'hybridper',
 'totalenroll',
 'lowincome',
 'white',
 'black',
 'hispanic',
 'asian',
      ]

final_data[rounding_cols] = final_data[rounding_cols].round(4)

In [23]:
# Convert columns to string first
final_data["schoolcode"] = final_data["schoolcode"].astype(str)
final_data['districtcode'] = final_data['districtcode'].astype(str)
final_data['countycode'] = final_data['countycode'].astype(str)

# Filter the data and create a copy to avoid SettingWithCopyWarning
first_export = final_data[final_data["year"] != 2022].copy()
first_export = first_export.drop("asian", axis=1)

# Add state information
first_export["state"] = "nyc"
first_export["districtcode"] = first_export["districtcode"] + 'nyc'
first_export["countycode"] = first_export["countycode"] + 'nyc'

# Finally, sort the data
final_data_sorted = first_export.sort_values(by=['schoolcode', 'year']).reset_index(drop=True)


In [24]:
# Identify the unique years in the dataset.
unique_years = set(final_data_sorted['year'])

# Group the data by ENTITY_CD and get the unique years for each group.
entity_groups = final_data_sorted.groupby('schoolcode')['year'].unique().reset_index()

# Find the ENTITY_CD values which have all the unique years.
valid_entity_cd = entity_groups[entity_groups['year'].apply(lambda x: set(x) == unique_years)]['schoolcode']

# Filter the main data for these ENTITY_CD values.
filtered_data = final_data_sorted[final_data_sorted['schoolcode'].isin(valid_entity_cd)]
filtered_data.shape

(3252, 15)

# Export NYC Data

In [25]:
filtered_data = filtered_data[filtered_data["year"] != 2022]

filtered_data.to_csv("../final_data_component/final_data_nyc_elapass.csv")

In [26]:
# final_data.to_csv("for_running.csv")